In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error , r2_score

import torch
import torch.nn as nn
import torch.optim as optim

# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


RuntimeError: Only a single TORCH_LIBRARY can be used to register the namespace prims; please put all of your definitions in a single TORCH_LIBRARY block.  If you were trying to specify implementations, consider using TORCH_LIBRARY_IMPL (which can be duplicated).  If you really intended to define operators for a single namespace in a distributed way, you can use TORCH_LIBRARY_FRAGMENT to explicitly indicate this.  Previous registration of TORCH_LIBRARY was registered at /dev/null:488; latest registration was registered at /dev/null:488

: 

In [5]:
# preparing the data: 
import csv

# Data extracted from the image
data = [
    [1, 0.2, 10000, 1500, 1.054],
    [2, 0.2, 10000, 2000, 0.76],
    [3, 0.2, 10000, 2500, 0.74],
    [4, 0.2, 15000, 1500, 0.72],
    [5, 0.2, 15000, 2000, 0.56],
    [6, 0.2, 15000, 2500, 0.74],
    [7, 0.2, 20000, 1500, 1.50],
    [8, 0.2, 20000, 2000, 1.14],
    [9, 0.2, 20000, 2500, 0.54],
    [10, 0.4, 10000, 1500, 0.68],
    [11, 0.4, 10000, 2000, 0.86],
    [12, 0.4, 10000, 2500, 0.72],
    [13, 0.4, 15000, 1500, 0.68],
    [14, 0.4, 15000, 2000, 0.72],
    [15, 0.4, 15000, 2500, 0.78],
    [16, 0.4, 20000, 1500, 0.64],
    [17, 0.4, 20000, 2000, 0.66],
    [18, 0.4, 20000, 2500, 0.52],
    [19, 0.6, 10000, 1500, 1.36],
    [20, 0.6, 10000, 2000, 1.10],
    [21, 0.6, 10000, 2500, 1.14],
    [22, 0.6, 15000, 1500, 0.92],
    [23, 0.6, 15000, 2000, 0.80],
    [24, 0.6, 15000, 2500, 0.84],
    [25, 0.6, 20000, 1500, 0.78],
    [26, 0.6, 20000, 2000, 0.76],
    [27, 0.6, 20000, 2500, 0.68],
]

# Write to CSV
with open("machining_data.csv", mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["Exp No.", "Ap (mm)", "Vc (tr/min)", "f (mm/tr)", "Ra (µm)"])  # headers
    writer.writerows(data)

print("CSV file 'machining_data.csv' created successfully.")


CSV file 'machining_data.csv' created successfully.


In [6]:

data = pd.read_csv("machining_data.csv" ,  encoding='latin1') 
data.head()


,Exp No.,Ap (mm),Vc (tr/min),f (mm/tr),Ra (µm)
0,1,0.2,10000,1500,1.054
1,2,0.2,10000,2000,0.760
2,3,0.2,10000,2500,0.740
3,4,0.2,15000,1500,0.720
4,5,0.2,15000,2000,0.560


In [4]:
data.info()
data.isnull().sum()
data.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Exp No.      27 non-null     int64  
 1   Ap (mm)      27 non-null     float64
 2   Vc (tr/min)  27 non-null     int64  
 3   f (mm/tr)    27 non-null     int64  
 4   Ra (µm)      27 non-null     float64
dtypes: float64(2), int64(3)
memory usage: 1.2 KB


,Exp No.,Ap (mm),Vc (tr/min),f (mm/tr),Ra (µm)
count,27.000000,27.00000,27.000000,27.000000,27.000000
mean,14.000000,0.40000,15000.000000,2000.000000,0.829407
std,7.937254,0.16641,4160.251472,416.025147,0.240829
min,1.000000,0.20000,10000.000000,1500.000000,0.520000
25%,7.500000,0.20000,10000.000000,1500.000000,0.680000
50%,14.000000,0.40000,15000.000000,2000.000000,0.760000
75%,20.500000,0.60000,20000.000000,2500.000000,0.890000
max,27.000000,0.60000,20000.000000,2500.000000,1.500000


# Base line ANN model 

In [7]:
# Replace 'target' with your actual target column name
X = data.drop(columns=["Ra (µm)"]).values
y = data["Ra (µm)"].values

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train, dtype=torch.float32).view(-1, 1).to(device)
y_test = torch.tensor(y_test, dtype=torch.float32).view(-1, 1).to(device)


NameError: name 'torch' is not defined

In [6]:
class BaselineANN(nn.Module):
    def __init__(self, input_dim):
        super(BaselineANN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 50),
            nn.ReLU(),
            nn.Linear(50, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )
        
    def forward(self, x):
        return self.net(x)

input_dim = X_train.shape[1]
model = BaselineANN(input_dim).to(device)
print(model)


BaselineANN(
  (net): Sequential(
    (0): Linear(in_features=4, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=1, bias=True)
  )
)


In [7]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [8]:
epochs = 50
train_losses = []

for epoch in range(epochs):
    model.train()
    
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    
    train_losses.append(loss.item())
    
    if (epoch + 1) % 10 == 0 or epoch == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")


Epoch [1/50], Loss: 0.9028
Epoch [10/50], Loss: 0.5783
Epoch [20/50], Loss: 0.3177
Epoch [30/50], Loss: 0.1324
Epoch [40/50], Loss: 0.0589
Epoch [50/50], Loss: 0.0497


In [ ]:
model.eval()
with torch.no_grad():
    y_pred = model(X_test).cpu().numpy()
    y_true = y_test.cpu().numpy()

mse = mean_squared_error(y_true, y_pred)
mae = mean_absolute_error(y_true, y_pred)
rmse = np.sqrt(mse)

print(f"📌 MSE: {mse:.4f}")
print(f"📌 MAE: {mae:.4f}")
print(f"📌 RMSE: {rmse:.4f}")
print(r2_score(y_true, y_pred))


📌 MSE: 0.0453
📌 MAE: 0.1916
📌 RMSE: 0.2128


## ✨ ANN with Xavier (Glorot) Initialization
We now define the same ANN structure, but manually initialize the weights using Xavier initialization.


In [10]:
class XavierANN(nn.Module):
    def __init__(self, input_dim):
        super(XavierANN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        
        self.relu = nn.ReLU()
        
        # Apply Xavier initialization
        nn.init.xavier_uniform_(self.fc1.weight)
        nn.init.xavier_uniform_(self.fc2.weight)
        nn.init.xavier_uniform_(self.fc3.weight)
        
        # Optional: Zero bias
        nn.init.zeros_(self.fc1.bias)
        nn.init.zeros_(self.fc2.bias)
        nn.init.zeros_(self.fc3.bias)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        return self.fc3(x)

# Instantiate
xavier_model = XavierANN(input_dim).to(device)


In [11]:
criterion = nn.MSELoss()
optimizer = optim.Adam(xavier_model.parameters(), lr=0.001)

epochs = 50
xavier_losses = []

for epoch in range(epochs):
    xavier_model.train()
    optimizer.zero_grad()
    outputs = xavier_model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    
    xavier_losses.append(loss.item())
    
    if (epoch + 1) % 10 == 0 or epoch == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")


Epoch [1/50], Loss: 0.8733
Epoch [10/50], Loss: 0.3845
Epoch [20/50], Loss: 0.1340
Epoch [30/50], Loss: 0.0746
Epoch [40/50], Loss: 0.0597
Epoch [50/50], Loss: 0.0392


In [12]:
xavier_model.eval()
with torch.no_grad():
    y_pred_xavier = xavier_model(X_test).cpu().numpy()
    y_true = y_test.cpu().numpy()

mse_xavier = mean_squared_error(y_true, y_pred_xavier)
mae_xavier = mean_absolute_error(y_true, y_pred_xavier)
rmse_xavier = np.sqrt(mse_xavier)

print(f"📌 Xavier - MSE: {mse_xavier:.4f}")
print(f"📌 Xavier - MAE: {mae_xavier:.4f}")
print(f"📌 Xavier - RMSE: {rmse_xavier:.4f}")


📌 Xavier - MSE: 0.0967
📌 Xavier - MAE: 0.2710
📌 Xavier - RMSE: 0.3109


## ⚡ ANN with He (Kaiming) Initialization
We now define the same ANN structure using He initialization, which is optimal for ReLU-activated layers.


In [13]:
class HeANN(nn.Module):
    def __init__(self, input_dim):
        super(HeANN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 50)
        self.fc2 = nn.Linear(50, 32)
        self.fc3 = nn.Linear(32, 1)
        
        self.relu = nn.ReLU()
        
        # Apply He (Kaiming) initialization
        nn.init.kaiming_uniform_(self.fc1.weight, nonlinearity='relu')
        nn.init.kaiming_uniform_(self.fc2.weight, nonlinearity='relu')
        nn.init.kaiming_uniform_(self.fc3.weight, nonlinearity='linear')  # output layer
        
        nn.init.zeros_(self.fc1.bias)
        nn.init.zeros_(self.fc2.bias)
        nn.init.zeros_(self.fc3.bias)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        return self.fc3(x)

# Instantiate
he_model = HeANN(input_dim).to(device)


In [14]:
criterion = nn.MSELoss()
optimizer = optim.Adam(he_model.parameters(), lr=0.001)

he_losses = []

for epoch in range(epochs):
    he_model.train()
    optimizer.zero_grad()
    outputs = he_model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    
    he_losses.append(loss.item())
    
    if (epoch + 1) % 10 == 0 or epoch == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")


Epoch [1/50], Loss: 0.2296
Epoch [10/50], Loss: 0.0440
Epoch [20/50], Loss: 0.0111
Epoch [30/50], Loss: 0.0064


Epoch [40/50], Loss: 0.0023
Epoch [50/50], Loss: 0.0009


In [15]:
he_model.eval()
with torch.no_grad():
    y_pred_he = he_model(X_test).cpu().numpy()
    y_true = y_test.cpu().numpy()

mse_he = mean_squared_error(y_true, y_pred_he)
mae_he = mean_absolute_error(y_true, y_pred_he)
rmse_he = np.sqrt(mse_he)

print(f"📌 He - MSE: {mse_he:.4f}")
print(f"📌 He - MAE: {mae_he:.4f}")
print(f"📌 He - RMSE: {rmse_he:.4f}")


📌 He - MSE: 0.1205
📌 He - MAE: 0.2827
📌 He - RMSE: 0.3471


# 🧠 CNN Model Variants for Regression
In this section, we’ll train CNN-based models on the `machining_data.csv` dataset.

### Experiments:
1. Baseline CNN
2. CNN with Xavier (Glorot) Initialization
3. CNN with He (Kaiming) Initialization

**Note**: We reshape the input as a 1D structure suitable for Conv1D.


In [16]:
# Assuming original shape is (batch_size, num_features)
# CNN expects (batch_size, channels, sequence_length)

X_train_cnn = X_train.unsqueeze(1)  # (batch, 1, features)
X_test_cnn = X_test.unsqueeze(1)


# Baseline CNN

In [17]:
class BaselineCNN(nn.Module):
    def __init__(self):
        super(BaselineCNN, self).__init__()
        self.conv1 = nn.Conv1d(1, 16, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv1d(16, 8, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(8 * X_train.shape[1], 1)

    def forward(self, x):
        x = self.relu1(self.conv1(x))
        x = self.relu2(self.conv2(x))
        x = self.flatten(x)
        return self.fc(x)

baseline_cnn = BaselineCNN().to(device)


In [18]:
criterion = nn.MSELoss()
optimizer = optim.Adam(baseline_cnn.parameters(), lr=0.001)

baseline_cnn_losses = []

for epoch in range(epochs):
    baseline_cnn.train()
    optimizer.zero_grad()
    outputs = baseline_cnn(X_train_cnn)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    baseline_cnn_losses.append(loss.item())

    if (epoch+1) % 10 == 0 or epoch == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")


Epoch 1/50, Loss: 0.4931
Epoch 10/50, Loss: 0.3439
Epoch 20/50, Loss: 0.2072
Epoch 30/50, Loss: 0.0980
Epoch 40/50, Loss: 0.0352
Epoch 50/50, Loss: 0.0296


In [19]:
baseline_cnn.eval()
with torch.no_grad():
    preds = baseline_cnn(X_test_cnn).cpu().numpy()
    targets = y_test.cpu().numpy()

mse_base = mean_squared_error(targets, preds)
mae_base = mean_absolute_error(targets, preds)
rmse_base = np.sqrt(mse_base)

print(f"Baseline CNN - MSE: {mse_base:.4f}, MAE: {mae_base:.4f}, RMSE: {rmse_base:.4f}")


Baseline CNN - MSE: 0.0464, MAE: 0.1775, RMSE: 0.2154


In [20]:
class XavierCNN(nn.Module):
    def __init__(self):
        super(XavierCNN, self).__init__()
        self.conv1 = nn.Conv1d(1, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(16, 8, kernel_size=3, padding=1)
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(8 * X_train.shape[1], 1)
        self.relu = nn.ReLU()

        # Xavier init
        nn.init.xavier_uniform_(self.conv1.weight)
        nn.init.xavier_uniform_(self.conv2.weight)
        nn.init.xavier_uniform_(self.fc.weight)
        nn.init.zeros_(self.conv1.bias)
        nn.init.zeros_(self.conv2.bias)
        nn.init.zeros_(self.fc.bias)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.flatten(x)
        return self.fc(x)

xavier_cnn = XavierCNN().to(device)


In [21]:
optimizer = optim.Adam(xavier_cnn.parameters(), lr=0.001)
xavier_cnn_losses = []

for epoch in range(epochs):
    xavier_cnn.train()
    optimizer.zero_grad()
    outputs = xavier_cnn(X_train_cnn)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    xavier_cnn_losses.append(loss.item())

    if (epoch+1) % 10 == 0 or epoch == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")


Epoch 1/50, Loss: 0.8685
Epoch 10/50, Loss: 0.6318


Epoch 20/50, Loss: 0.4443
Epoch 30/50, Loss: 0.2870
Epoch 40/50, Loss: 0.1641
Epoch 50/50, Loss: 0.0866


In [22]:
xavier_cnn.eval()
with torch.no_grad():
    preds = xavier_cnn(X_test_cnn).cpu().numpy()

mse_xav = mean_squared_error(targets, preds)
mae_xav = mean_absolute_error(targets, preds)
rmse_xav = np.sqrt(mse_xav)

print(f"Xavier CNN - MSE: {mse_xav:.4f}, MAE: {mae_xav:.4f}, RMSE: {rmse_xav:.4f}")


Xavier CNN - MSE: 0.1332, MAE: 0.3103, RMSE: 0.3649


In [23]:
class HeCNN(nn.Module):
    def __init__(self):
        super(HeCNN, self).__init__()
        self.conv1 = nn.Conv1d(1, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(16, 8, kernel_size=3, padding=1)
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(8 * X_train.shape[1], 1)
        self.relu = nn.ReLU()

        # He (Kaiming) init
        nn.init.kaiming_uniform_(self.conv1.weight, nonlinearity='relu')
        nn.init.kaiming_uniform_(self.conv2.weight, nonlinearity='relu')
        nn.init.kaiming_uniform_(self.fc.weight, nonlinearity='linear')
        nn.init.zeros_(self.conv1.bias)
        nn.init.zeros_(self.conv2.bias)
        nn.init.zeros_(self.fc.bias)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.flatten(x)
        return self.fc(x)

he_cnn = HeCNN().to(device)


In [24]:
optimizer = optim.Adam(he_cnn.parameters(), lr=0.001)
he_cnn_losses = []

for epoch in range(epochs):
    he_cnn.train()
    optimizer.zero_grad()
    outputs = he_cnn(X_train_cnn)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    he_cnn_losses.append(loss.item())

    if (epoch+1) % 10 == 0 or epoch == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")


Epoch 1/50, Loss: 0.8338
Epoch 10/50, Loss: 0.3688
Epoch 20/50, Loss: 0.1400
Epoch 30/50, Loss: 0.0605
Epoch 40/50, Loss: 0.0403
Epoch 50/50, Loss: 0.0333


In [25]:
he_cnn.eval()
with torch.no_grad():
    preds = he_cnn(X_test_cnn).cpu().numpy()

mse_he = mean_squared_error(targets, preds)
mae_he = mean_absolute_error(targets, preds)
rmse_he = np.sqrt(mse_he)

print(f"He CNN - MSE: {mse_he:.4f}, MAE: {mae_he:.4f}, RMSE: {rmse_he:.4f}")


He CNN - MSE: 0.0918, MAE: 0.2186, RMSE: 0.3029


In [26]:
full_data = pd.read_csv("data.csv")
full_data.head()

,Angle d'inclinaison( ɸ),Vitesse S (tr/min),l'avance Fy (mm/min),L'avance Fx (mm),Profondeur de passe t (mm),Moyenne Ra (Expiremental),Rugosité prédite ANN (BAR)(µm),Rugosité prédite ANN&GA (µm),Poids prédite GA,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,0,380,22,0.4,0.2,1.36,1.362014,1.573254,2.999688,0.002014,0.000004,NaN,NaN
1,0,380,34,0.6,0.2,2.11,2.442357,2.995118,3.596372,0.332357,0.110461,NaN,NaN
2,0,380,22,0.4,0.4,1.95,2.949477,1.299246,2.436648,0.999477,0.998953,NaN,NaN
3,0,380,34,0.6,0.4,3.55,3.546757,2.562542,-0.343223,-0.003243,0.000011,NaN,1.102973
4,0,380,22,0.4,0.6,0.56,0.547630,1.006231,-0.762090,-0.012370,0.000153,NaN,NaN


In [27]:
df = full_data[["Angle d'inclinaison( ɸ)", "Vitesse S (tr/min)", "Profondeur de passe t (mm)", "Moyenne Ra (Expiremental)"]]
df.head()



,Angle d'inclinaison( ɸ),Vitesse S (tr/min),Profondeur de passe t (mm),Moyenne Ra (Expiremental)
0,0,380,0.2,1.36
1,0,380,0.2,2.11
2,0,380,0.4,1.95
3,0,380,0.4,3.55
4,0,380,0.6,0.56


In [ ]:
data.head()

,Exp No.,Ap (mm),Vc (tr/min),f (mm/tr),Ra (µm)
0,1,0.2,10000,1500,1.054
1,2,0.2,10000,2000,0.760
2,3,0.2,10000,2500,0.740
3,4,0.2,15000,1500,0.720
4,5,0.2,15000,2000,0.560


: 